In [8]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
import time
import numpy as np
import pandas as pd

from sklearn.utils import shuffle

from tensorflow.keras.backend import backend
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, SimpleRNN, Dropout
#from keras.layers.embeddings import Embedding

print ("============================================================")
print (" RNN for demographic features          Muratova Anna        ")
print ("============================================================")

start_time = time.time()

df = pd.read_excel("table.xlsx")

df.dropna(inplace = True)   # remove empty (NaN) lines
df = shuffle(df, random_state=7)
print ("Input data shape =", df.shape)
size = df.shape[0]
t = int(size * 0.8)         # train part size

x = df.as_matrix()[:size, 0:6]
y = df.as_matrix()[:size, 6] 
del df

X = np.empty((x.shape[0], x.shape[1]), dtype='int32')

for d in range(x.shape[1]):        
    xs = list(set(x[:, d]))
    if d == 0:
        print ("Number of unique sequences =", len(xs), "\n")
    xd = {xs[i]: i for i in range(len(xs))}
    for l in range(x.shape[0]):
        X[l][d] = xd[x[l][d]]

#X = StandardScaler().fit_transform(X)

XF_train = X[0:t, 1:]           # features only
XF_test  = X[t:size, 1:]
#print XF_train[0:10]

yn = np.empty((size), dtype='int32')

ys = list(set(y[:]))
yd = {ys[i]: i for i in range(len(ys))}
for l in range(len(yn)):
        yn[l] = yd[y[l]]


y_train = yn[0:t]
y_test  = yn[t:size]

# all unique characters to the set
events = set()
for seq in x[:, 0]:
    for event in seq:
        events.add(event)

events = list(events)
#print events

event_to_id = {t:i+1 for i,t in enumerate(events)}
#print event_to_id

max_seq_len = 8
seq_events_numbered = np.zeros((x.shape[0], max_seq_len), dtype='int32')

for i in range(seq_events_numbered.shape[0]):
    for k in range(len(x[i][0])):
        seq_events_numbered[i][k] = event_to_id[x[i][0][k]]

S_train = seq_events_numbered[0:t, :]       # train sequences
S_test  = seq_events_numbered[t:size, :]    # test  sequences


print ("Train features  data shape =", XF_train.shape, y_train.shape)
print ("Test  features  data shape =", XF_test.shape,  y_test.shape)
print ("\nData preprocessing time, sec =  %0.2f" % (time.time() - start_time))

print ("============================================================")

print ("Keras backend =", backend())
 
time_01 = time.time()
print ("\nRNN classification by features") 

features = Sequential()
features.add(Dense(100, activation="sigmoid", input_dim=5))
features.add(Dropout(0.1))
features.add(Dense(100, activation='sigmoid'))
features.add(Dense(1, activation="sigmoid"))
features.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
features.fit(XF_train, y_train, epochs=400, batch_size=100)

time_02 = time.time()
score, acc = features.evaluate(XF_test, y_test, verbose=0)

time_03 = time.time()

print ("\tModel fitting time .... %0.2f" % (time_02 - time_01))
print ("\tPrediction time ....... %0.2f" % (time_03 - time_02))
print ("\tTotal time ............ %0.2f" % (time_03 - time_01))    
print ("Accuracy: %.3f" % (acc))
print ("============================================================")


 RNN for demographic features          Muratova Anna        
Input data shape = (6626, 7)
Number of unique sequences = 1228 

Train features  data shape = (5300, 5) (5300,)
Test  features  data shape = (1326, 5) (1326,)

Data preprocessing time, sec =  0.39
Keras backend = tensorflow

RNN classification by features
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


/home/alexander/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/alexander/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Epoch 1/400
5300/5300 [==============================] - 1s 102us/sample - loss: 0.7382 - acc: 0.5174
Epoch 2/400
5300/5300 [==============================] - 0s 18us/sample - loss: 0.6909 - acc: 0.5319
Epoch 3/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6873 - acc: 0.5564
Epoch 4/400
5300/5300 [==============================] - 0s 18us/sample - loss: 0.6834 - acc: 0.5685
Epoch 5/400
5300/5300 [==============================] - 0s 18us/sample - loss: 0.6780 - acc: 0.5877
Epoch 6/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6744 - acc: 0.5919
Epoch 7/400
5300/5300 [==============================] - 0s 18us/sample - loss: 0.6720 - acc: 0.5908
Epoch 8/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6678 - acc: 0.5940
Epoch 9/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6650 - acc: 0.6019
Epoch 10/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6641 - 

5300/5300 [==============================] - 0s 17us/sample - loss: 0.6601 - acc: 0.6123
Epoch 82/400
5300/5300 [==============================] - 0s 19us/sample - loss: 0.6597 - acc: 0.6108
Epoch 83/400
5300/5300 [==============================] - 0s 18us/sample - loss: 0.6585 - acc: 0.6125
Epoch 84/400
5300/5300 [==============================] - 0s 20us/sample - loss: 0.6597 - acc: 0.6079
Epoch 85/400
5300/5300 [==============================] - 0s 18us/sample - loss: 0.6588 - acc: 0.6204
Epoch 86/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6604 - acc: 0.6096
Epoch 87/400
5300/5300 [==============================] - 0s 19us/sample - loss: 0.6602 - acc: 0.6087
Epoch 88/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6595 - acc: 0.6164
Epoch 89/400
5300/5300 [==============================] - 0s 20us/sample - loss: 0.6590 - acc: 0.6121
Epoch 90/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6601 - acc: 

5300/5300 [==============================] - 0s 16us/sample - loss: 0.6558 - acc: 0.6189
Epoch 161/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6570 - acc: 0.6149
Epoch 162/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6561 - acc: 0.6196
Epoch 163/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6570 - acc: 0.6130
Epoch 164/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6563 - acc: 0.6217
Epoch 165/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6570 - acc: 0.6157
Epoch 166/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6560 - acc: 0.6168
Epoch 167/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6563 - acc: 0.6160
Epoch 168/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6560 - acc: 0.6206
Epoch 169/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.655

5300/5300 [==============================] - 0s 16us/sample - loss: 0.6531 - acc: 0.6228
Epoch 240/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6538 - acc: 0.6232
Epoch 241/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6539 - acc: 0.6198
Epoch 242/400
5300/5300 [==============================] - 0s 18us/sample - loss: 0.6529 - acc: 0.6228
Epoch 243/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6530 - acc: 0.6251
Epoch 244/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6540 - acc: 0.6202
Epoch 245/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6532 - acc: 0.6257
Epoch 246/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6536 - acc: 0.6213
Epoch 247/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6542 - acc: 0.6196
Epoch 248/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.653

5300/5300 [==============================] - 0s 17us/sample - loss: 0.6499 - acc: 0.6219
Epoch 319/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6497 - acc: 0.6228
Epoch 320/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6493 - acc: 0.6255
Epoch 321/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6490 - acc: 0.6247
Epoch 322/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6504 - acc: 0.6253
Epoch 323/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6489 - acc: 0.6245
Epoch 324/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6489 - acc: 0.6240
Epoch 325/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6488 - acc: 0.6253
Epoch 326/400
5300/5300 [==============================] - 0s 17us/sample - loss: 0.6489 - acc: 0.6260
Epoch 327/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.649

5300/5300 [==============================] - 0s 18us/sample - loss: 0.6459 - acc: 0.6262
Epoch 398/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6476 - acc: 0.6266
Epoch 399/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6467 - acc: 0.6253
Epoch 400/400
5300/5300 [==============================] - 0s 16us/sample - loss: 0.6480 - acc: 0.6262
	Model fitting time .... 38.76
	Prediction time ....... 0.08
	Total time ............ 38.84
Accuracy: 0.630
